In [5]:
# SETUP CELL - RUN THIS FIRST!
# Put all imports here so you can easily re-run them

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pickle
import warnings
warnings.filterwarnings('ignore')

# Set up paths that you'll use throughout
data_dir = Path('../data')

print("✅ Setup complete!")
print(f"   Pandas version: {pd.__version__}")
print(f"   Data directory: {data_dir.absolute()}")

✅ Setup complete!
   Pandas version: 2.3.1
   Data directory: /home/katiecho45/xqtl-protocol/code/xqtl_modifier_score/cv2f/notebooks/../data


In [1]:
# First cell - test imports
try:
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from pathlib import Path
    print("✅ All packages imported successfully!")
    print(f"Pandas version: {pd.__version__}")
    print(f"Numpy version: {np.__version__}")
except ImportError as e:
    print(f"❌ Missing package: {e}")
    print("Install with: pip install pandas numpy matplotlib seaborn")


✅ All packages imported successfully!
Pandas version: 2.3.1
Numpy version: 2.2.6


In [2]:
# Set up the path to your data
from pathlib import Path

# Since notebook is in 'notebooks' folder, go up one level to find 'data'
data_dir = Path('../data')

# Check if path is correct
if data_dir.exists():
    print("✅ Found data directory!")
    print(f"Full path: {data_dir.absolute()}")
else:
    print("❌ Data directory not found. Checking current location...")
    print(f"Current directory: {Path.cwd()}")
    print("Contents here:", list(Path.cwd().parent.iterdir()))

✅ Found data directory!
Full path: /home/katiecho45/xqtl-protocol/code/xqtl_modifier_score/cv2f/notebooks/../data


In [3]:
# List all your data files
print("📁 YOUR DATA FILES:")
print("=" * 50)

for item in sorted(data_dir.iterdir()):
    if item.is_dir():
        print(f"\n📁 {item.name}/")
        # Show first 5 files in each folder
        sub_items = list(item.iterdir())[:5]
        for sub in sub_items:
            print(f"   - {sub.name}")
        if len(list(item.iterdir())) > 5:
            print(f"   ... and {len(list(item.iterdir())) - 5} more files")
    else:
        print(f"📄 {item.name}")

📁 YOUR DATA FILES:

📁 MLxQTL/
   - MLxQTL-main
📄 README.md

📁 annotated_data_Ast_mega_eQTL_chr2.parquet/
   - part.1.parquet
   - part.2.parquet
   - part.3.parquet
   - part.0.parquet

📁 cv2f_files/
   - 1000G_BIMS_hg38
   - positive_set.PIP0.7_CS0_VCP0.8_COS0.txt
   - LAVA_LDblocks_published.txt
   - MAF_features_Aug032022.txt
   - negative_set.PIP0.7_CS0_VCP0.8_COS0.txt

📁 no_xqtl_features_v2f/
   - no_xqtl_features_v2f.18.txt.gz
   - no_xqtl_features_v2f.7.txt.gz
   - no_xqtl_features_v2f.9.txt.gz
   - no_xqtl_features_v2f.4.txt.gz
   - no_xqtl_features_v2f.20.txt.gz
   ... and 17 more files
📄 part.0.parquet

📁 sampledata/
   - test_NPR_10_PIP_0.9_0.01
   - train_NPR_10_PIP_0.1_0.01
   - columns_dict.pkl
   - 41588_2024_1820_MOESM4_ESM.xlsx
   - model_standard_subset_conservative_weighted_chr_chr2_NPR_10.joblib
   ... and 1 more files

📁 xqtl_toploci/
   - fine_mapping
   - colocboost


In [4]:
# Load pre-selected variants
print("LOADING POSITIVE AND NEGATIVE SETS")
print("=" * 50)

# Positive set (disease-causing variants)
positive_file = data_dir / 'cv2f_files' / 'positive_set.PIP0.7_CS0_VCP0.8_COS0.txt'
positive_set = pd.read_csv(positive_file, sep='\t')
print(f"✅ Positive set: {len(positive_set)} variants")

# Negative set (non-disease variants)  
negative_file = data_dir / 'cv2f_files' / 'negative_set.PIP0.7_CS0_VCP0.8_COS0.txt'
negative_set = pd.read_csv(negative_file, sep='\t')
print(f"✅ Negative set: {len(negative_set)} variants")

print(f"\nRatio: {len(negative_set)/len(positive_set):.1f} negative for every 1 positive")

LOADING POSITIVE AND NEGATIVE SETS
✅ Positive set: 1786 variants
✅ Negative set: 16190724 variants

Ratio: 9065.4 negative for every 1 positive


In [5]:
# See what information we have about each variant
print("POSITIVE SET STRUCTURE:")
print("Columns:", list(positive_set.columns))
print("\nFirst 3 variants:")
positive_set.head(3)

POSITIVE SET STRUCTURE:
Columns: ['rs12037841']

First 3 variants:


,rs12037841
0,rs6661489
1,rs7515905
2,rs1752684


In [6]:
# See example negative variants
print("NEGATIVE SET STRUCTURE:")
print("Columns:", list(negative_set.columns))
print("\nFirst 3 variants:")
negative_set.head(3)

NEGATIVE SET STRUCTURE:
Columns: ['rs112409036']

First 3 variants:


,rs112409036
0,rs11166389
1,rs116740877
2,rs550182761


In [7]:
# Summary of what you found
print("=" * 60)
print("SUMMARY - WHAT YOU DISCOVERED:")
print("=" * 60)
print(f"""
✅ Data successfully loaded!

You have:
- {len(positive_set):,} positive variants (likely cause Alzheimer's)
- {len(negative_set):,} negative variants (don't cause Alzheimer's)
- Ratio: {len(negative_set)/len(positive_set):.1f}:1 (negative:positive)

Columns available: {len(positive_set.columns)} features per variant

Next: Analyze what features distinguish positive from negative variants
""")

SUMMARY - WHAT YOU DISCOVERED:

✅ Data successfully loaded!

You have:
- 1,786 positive variants (likely cause Alzheimer's)
- 16,190,724 negative variants (don't cause Alzheimer's)
- Ratio: 9065.4:1 (negative:positive)

Columns available: 1 features per variant

Next: Analyze what features distinguish positive from negative variants



In [8]:
# The files have headers that aren't being read correctly
# Let's investigate the file structure

print("INVESTIGATING FILE STRUCTURE...")
print("=" * 50)

# Look at the raw content of positive file
positive_file = data_dir / 'cv2f_files' / 'positive_set.PIP0.7_CS0_VCP0.8_COS0.txt'
print("First 5 lines of positive set file:")
with open(positive_file, 'r') as f:
    for i in range(5):
        line = f.readline()
        print(f"Line {i}: {line[:100]}...")  # First 100 chars of each line

INVESTIGATING FILE STRUCTURE...
First 5 lines of positive set file:
Line 0: rs12037841
...
Line 1: rs6661489
...
Line 2: rs7515905
...
Line 3: rs1752684
...
Line 4: rs679515
...


In [9]:
# Try different loading approaches
print("TRYING DIFFERENT LOADING METHODS...")
print("=" * 50)

# Method 1: No header
try:
    positive_set_v1 = pd.read_csv(positive_file, sep='\t', header=None)
    print(f"Method 1 (no header): {positive_set_v1.shape}")
    print("First row:", positive_set_v1.iloc[0].tolist()[:5])
except Exception as e:
    print(f"Method 1 failed: {e}")

# Method 2: Different separator
try:
    positive_set_v2 = pd.read_csv(positive_file, sep='\s+', header=None)
    print(f"\nMethod 2 (whitespace separator): {positive_set_v2.shape}")
    print("First row:", positive_set_v2.iloc[0].tolist()[:5])
except Exception as e:
    print(f"Method 2 failed: {e}")

# Method 3: As single column
try:
    with open(positive_file, 'r') as f:
        lines = f.readlines()
    print(f"\nMethod 3 (raw read): {len(lines)} lines")
    print("First 5 variant IDs:", [line.strip() for line in lines[:5]])
except Exception as e:
    print(f"Method 3 failed: {e}")

TRYING DIFFERENT LOADING METHODS...
Method 1 (no header): (1787, 1)
First row: ['rs12037841']

Method 2 (whitespace separator): (1787, 1)
First row: ['rs12037841']

Method 3 (raw read): 1787 lines
First 5 variant IDs: ['rs12037841', 'rs6661489', 'rs7515905', 'rs1752684', 'rs679515']


In [10]:
# Let's look at the other data files that might have the features
print("EXPLORING OTHER DATA SOURCES...")
print("=" * 50)

# Check MAF features file
maf_file = data_dir / 'cv2f_files' / 'MAF_features_Aug032022.txt'
try:
    maf_df = pd.read_csv(maf_file, sep='\t', nrows=5)
    print("MAF Features file:")
    print(f"  Shape: {maf_df.shape}")
    print(f"  Columns: {list(maf_df.columns)}")
except Exception as e:
    print(f"MAF file error: {e}")

# Check the columns dictionary
columns_file = data_dir / 'sampledata' / 'columns_dict.pkl'
try:
    with open(columns_file, 'rb') as f:
        columns_dict = pickle.load(f)
    print(f"\nModel features from columns_dict:")
    print(f"  Total features: {len(columns_dict)}")
    print(f"  First 10 features: {list(columns_dict.keys())[:10]}")
except Exception as e:
    print(f"Columns dict error: {e}")

# Check astrocyte parquet
ast_file = data_dir / 'annotated_data_Ast_mega_eQTL_chr2.parquet'
try:
    ast_df = pd.read_parquet(ast_file)
    print(f"\nAstrocyte eQTL data:")
    print(f"  Shape: {ast_df.shape}")
    print(f"  Columns: {list(ast_df.columns)[:10]}...")
except Exception as e:
    print(f"Astrocyte file error: {e}")

EXPLORING OTHER DATA SOURCES...
MAF Features file:
  Shape: (5, 6)
  Columns: ['CHR', 'SNP', 'A1', 'A2', 'MAF', 'NCHROBS']
Columns dict error: name 'pickle' is not defined

Astrocyte eQTL data:
  Shape: (9625, 4946)
  Columns: ['variant_id', 'gene_id', 'chr', 'pos', 'ref', 'alt', 'pip', 'distance_TSS', 'abs_distance_TSS', 'abs_distance_TSS_log']...


In [2]:
# Fix: import pickle first
import pickle

# Now re-read the data
print("UNDERSTANDING THE DATA STRUCTURE")
print("=" * 50)

# Make sure data_dir is defined (re-run this if needed)
from pathlib import Path
data_dir = Path('../data')

# Read positive variants as a list
with open(data_dir / 'cv2f_files' / 'positive_set.PIP0.7_CS0_VCP0.8_COS0.txt', 'r') as f:
    positive_ids = [line.strip() for line in f.readlines()]

with open(data_dir / 'cv2f_files' / 'negative_set.PIP0.7_CS0_VCP0.8_COS0.txt', 'r') as f:
    negative_ids = [line.strip() for line in f.readlines()]

print(f"✅ Positive variant IDs: {len(positive_ids)}")
print(f"✅ Negative variant IDs: {len(negative_ids)}")
print(f"Ratio: {len(negative_ids)/len(positive_ids):.1f}:1")
print(f"\nExample positive IDs: {positive_ids[:5]}")
print(f"Example negative IDs: {negative_ids[:5]}")

# Key insight
print("\n" + "="*50)
print("KEY INSIGHT:")
print("The positive/negative files are LISTS of variant IDs.")
print("The actual features are in the parquet files (Astrocyte data has 4946 features!).")
print("Your job: Understand HOW these IDs were selected!")

UNDERSTANDING THE DATA STRUCTURE
✅ Positive variant IDs: 1787
✅ Negative variant IDs: 16190725
Ratio: 9060.3:1

Example positive IDs: ['rs12037841', 'rs6661489', 'rs7515905', 'rs1752684', 'rs679515']
Example negative IDs: ['rs112409036', 'rs11166389', 'rs116740877', 'rs550182761', 'rs549210756']

KEY INSIGHT:
The positive/negative files are LISTS of variant IDs.
The actual features are in the parquet files (Astrocyte data has 4946 features!).
Your job: Understand HOW these IDs were selected!


In [6]:
# Make sure all imports are available
import pandas as pd
import numpy as np
from pathlib import Path

# Re-define data_dir if needed
data_dir = Path('../data')

# Now run the matching code
print("MATCHING VARIANTS WITH FEATURES")
print("=" * 50)

# Load the Astrocyte data (which has features)
ast_df = pd.read_parquet(data_dir / 'annotated_data_Ast_mega_eQTL_chr2.parquet')
print(f"Astrocyte data: {ast_df.shape[0]} variants with {ast_df.shape[1]} features")

# Check if our positive/negative variants are in this dataset
ast_variant_ids = set(ast_df['variant_id'].values)
positive_in_ast = [v for v in positive_ids if v in ast_variant_ids]
negative_in_ast = [v for v in negative_ids[:1000] if v in ast_variant_ids]  # Check first 1000

print(f"\nPositive variants in Astrocyte data: {len(positive_in_ast)} / {len(positive_ids)}")
print(f"Negative variants in Astrocyte data (first 1000): {len(negative_in_ast)} / 1000")

if len(positive_in_ast) > 0:
    # Show features for first positive variant
    example_variant = positive_in_ast[0]
    example_features = ast_df[ast_df['variant_id'] == example_variant].iloc[0]
    print(f"\nExample: Features for positive variant {example_variant}:")
    print(f"  Chromosome: {example_features['chr']}")
    print(f"  Position: {example_features['pos']}")
    print(f"  PIP score: {example_features['pip']:.4f}")
    print(f"  Distance to TSS: {example_features['distance_TSS']}")

MATCHING VARIANTS WITH FEATURES
Astrocyte data: 9625 variants with 4946 features

Positive variants in Astrocyte data: 0 / 1787
Negative variants in Astrocyte data (first 1000): 0 / 1000


In [7]:
print("=" * 60)
print("🎯 WHAT YOU'VE DISCOVERED:")
print("=" * 60)
print(f"""
1. DATA STRUCTURE:
   - Positive set: {len(positive_ids):,} variant IDs (just IDs, no features)
   - Negative set: {len(negative_ids):,} variant IDs (massive set!)
   - Ratio: {len(negative_ids)/len(positive_ids):.0f}:1
   
2. FEATURES AVAILABLE:
   - Astrocyte eQTL data: {ast_df.shape[1]} features per variant
   - These include: PIP scores, gene expression, distances, etc.
   
3. SELECTION CRITERIA (from filenames):
   - PIP ≥ 0.7 (70% probability variant is causal)
   - CS0 (in credible set 0 - highest confidence)
   - VCP ≥ 0.8 (80% variant causal probability)
   
4. YOUR TASK:
   - Understand WHY these specific {len(positive_ids)} variants were chosen as positive
   - Understand WHY these specific {len(negative_ids):,} variants were chosen as negative
   - Create your own selection logic based on biological reasoning
   
Next step: Analyze what features distinguish positive from negative variants!
""")

🎯 WHAT YOU'VE DISCOVERED:

1. DATA STRUCTURE:
   - Positive set: 1,787 variant IDs (just IDs, no features)
   - Negative set: 16,190,725 variant IDs (massive set!)
   - Ratio: 9060:1

2. FEATURES AVAILABLE:
   - Astrocyte eQTL data: 4946 features per variant
   - These include: PIP scores, gene expression, distances, etc.

3. SELECTION CRITERIA (from filenames):
   - PIP ≥ 0.7 (70% probability variant is causal)
   - CS0 (in credible set 0 - highest confidence)
   - VCP ≥ 0.8 (80% variant causal probability)

4. YOUR TASK:
   - Understand WHY these specific 1787 variants were chosen as positive
   - Understand WHY these specific 16,190,725 variants were chosen as negative
   - Create your own selection logic based on biological reasoning

Next step: Analyze what features distinguish positive from negative variants!

